<a href="https://colab.research.google.com/github/mayur0408/supervised_learning_practice/blob/main/Fake_news_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
sample = pd.read_csv('/content/train.csv')
sample.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
sample = sample.drop(['id'],axis=1)

In [ ]:
sample.shape

(20800, 4)

In [ ]:
sample.dropna(inplace=True)

In [ ]:
sample.shape

(18285, 4)

In [ ]:
sample.reset_index(inplace=True)

In [ ]:
X=sample.drop(['label'],axis=1)
X.head()

,index,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [ ]:
Y=sample['label']
Y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [ ]:
import re 
import nltk 
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
pt=PorterStemmer()
lematize=WordNetLemmatizer()
corpus = []
for i in range(0,len(X)):
  sent = re.sub('[^a-zA-Z]',' ',X['title'][i])
  sent = sent.lower()
  sent = sent.split()
  sent = [pt.stem(word) for word in sent if not word in set(stopwords.words('english'))]
  sent = ' '.join(sent)
  corpus.append(sent)


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
cv=CountVectorizer(max_features=5000,ngram_range=(1,3))
# cv=TfidfVectorizer(max_features=5000,ngram_range=(1,3))
X=cv.fit_transform(corpus).toarray()
X.shape

(18285, 5000)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.3,random_state=42)
print(f'{len(X_train)} {len(Y_train)}')
print(f'{len(X_test)} {len(Y_test)}')

12799 12799
5486 5486


In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import PassiveAggressiveClassifier
# model=MultinomialNB()
model=PassiveAggressiveClassifier(C = 0.3, loss = 'squared_hinge' , max_iter = 1500)
model.fit(X_train,Y_train)

PassiveAggressiveClassifier(C=0.3, average=False, class_weight=None,
                            early_stopping=False, fit_intercept=True,
                            loss='squared_hinge', max_iter=1500,
                            n_iter_no_change=5, n_jobs=None, random_state=None,
                            shuffle=True, tol=0.001, validation_fraction=0.1,
                            verbose=0, warm_start=False)

In [ ]:
Y_pred=model.predict(X_test)
from sklearn.metrics import confusion_matrix,accuracy_score
cm=confusion_matrix(Y_test,Y_pred)
print(cm)
score=accuracy_score(Y_test,Y_pred)
print(score)

[[2840  267]
 [ 218 2161]]
0.9115931461903026


In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold,cross_val_score
cv=StratifiedKFold(n_splits=10,shuffle=True,random_state=0)
score = cross_val_score(model,X,Y,scoring='accuracy',cv=cv)
print(max(score))
print(min(score))
print(np.mean(score))

0.9256424275560415
0.9053610503282276
0.916324192172547


In [ ]:
# from sklearn.model_selection import RandomizedSearchCV,KFold
# params = {'C':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.9,1],
#           'max_iter':[500,1000,1100,1200,1300,1400,1500,2000],
#           'loss':['hinge','squared_hinge']
# }
# cv=KFold(n_splits=5,shuffle=True)
# rv=RandomizedSearchCV(model,param_distributions=params,cv=cv,verbose=3)
# rv.fit(X,Y)
# best_params=rv.best_params_
# best_score=rv.best_score_

# 'C': 0.3, 'loss': 'squared_hinge', 'max_iter': 1500
# 0.9195515449822258

In [ ]:
best_params

{'C': 0.3, 'loss': 'squared_hinge', 'max_iter': 1500}

In [ ]:
best_score

0.9195515449822258